In [1]:
from mxnet import nd
from mxnet.gluon import nn
import pandas as pd
import mxnet as mx
import time
data_ctx = mx.gpu()
model_ctx = mx.gpu()        
mx.random.seed(42, ctx=model_ctx)

In [2]:
# loading the dataset using mx.test_utils
mnist = mx.test_utils.get_mnist()
batch_size = 100
learning_rate = 0.1

# Convert training and validation data to NDArray format
train_data_array = mx.nd.array(mnist['train_data'])
train_label_array = mx.nd.array(mnist['train_label'])
test_data_array = mx.nd.array(mnist['test_data'])
test_label_array = mx.nd.array(mnist['test_label'])

# Combine training and validation data
combined_data = mx.nd.concatenate([train_data_array, test_data_array], axis=0)
combined_label = mx.nd.concatenate([train_label_array, test_label_array], axis=0)

# Create an iterator with combined data
combined_data_iter = mx.io.NDArrayIter(combined_data, combined_label, batch_size, shuffle=True)

In [3]:
# defining a neural net
net = nn.Sequential()
with net.name_scope():
    net.add(nn.Dense(units=200, activation='relu', use_bias=True, dtype='float32', in_units=784))
    net.add(nn.Dense(units=100, activation='relu', use_bias=True, dtype='float32', in_units=200))
    net.add(nn.Dense(units=10, activation=None, use_bias=True, dtype='float32', in_units=100))

In [4]:
net.initialize(mx.initializer.Uniform(), ctx=model_ctx)

In [5]:
trainer = mx.gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': learning_rate})

In [6]:
num_of_epochs = 10
metric = mx.metric.Accuracy()
loss = mx.gluon.loss.SoftmaxCrossEntropyLoss()

In [7]:
start = time.time()

In [8]:
for i in range(num_of_epochs):
    combined_data_iter.reset()
    for batch in combined_data_iter:
        data = mx.gluon.utils.split_and_load(batch.data[0], ctx_list=[data_ctx], batch_axis=0)
        label = mx.gluon.utils.split_and_load(batch.label[0], ctx_list=[data_ctx], batch_axis=0)
        outputs = []
        with mx.autograd.record():
            for x, y in zip(data, label):
                z = net(x)
                loss_value = loss(z, y)
                loss_value.backward()
                outputs.append(z)
        metric.update(label, outputs)
        trainer.step(batch.data[0].shape[0])
    metric.reset()

In [9]:
end = time.time()

In [10]:
print(f"Time taken to train the model: {end - start} seconds")

Time taken to train the model: 19.209173679351807 seconds
